<a href="https://colab.research.google.com/github/sarasabino/Sistema_Recomendacion_BBDD_Explotaciones/blob/main/MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - BBDD Generation recommender system

One of the main difficulties on the implementation of this project is the fact that the data needs to be standarized in order to be able to compare. 

For the purpose to present the idea in a clear way this notebook present a simple example of how the recommender system will proceed once the data is all standarized. 

This problem with the data is not going to be present once this previous work is done because once the project is in "pro" version the user will feed the system with their answers in a standarized way. It will be only be relevant to keep going the recommender as first place. 

The data use in this notebook is totally manipulated to be able to compare at first without doing the standarizing exercise as it will be taking long.

In [71]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os
import time

In [72]:


! cp /content/drive/MyDrive/projects_with_tables.csv .



In [73]:
test_file = pd.read_csv('projects_with_tables.csv', sep = ';', header=None)

In [74]:
test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,RRHH,Plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brecha Salarial,Plantilla_mensual,Brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RRHH,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,RRHH,Entidades RRHH,Indicadores RRHH,Periodos Informes RRHH,Valores Informe RRHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RRHH,Indicadores RSC,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,Responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_MES,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades


In [75]:
#We set all text in the df to lowercase
test_file = test_file.apply(lambda x : x.str.lower())

test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,rrhh,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brecha salarial,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rrhh,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,rrhh,entidades rrhh,indicadores rrhh,periodos informes rrhh,valores informe rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rrhh,indicadores rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades


In [76]:
#sustuimos todos los espacios de los nombres de las tablas pcon _

for column in test_file.columns:
    test_file[column] = test_file[column].str.replace(' ', '_')

In [77]:
test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,rrhh,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brecha_salarial,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rrhh,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,rrhh,entidades_rrhh,indicadores_rrhh,periodos_informes_rrhh,valores_informe_rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rrhh,indicadores_rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades


In [78]:
#We create a new column with a list containing all tables in that project

test_file['tables'] = test_file.loc[:,test_file.columns != 0].values.tolist()

test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,tables
0,rrhh,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[plantilla_mensual, tipo_contrato, categoria_p..."
1,brecha_salarial,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[plantilla_mensual, brecha_salarios, tipo_cont..."
2,rrhh,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN,"[rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh..."
3,rrhh,entidades_rrhh,indicadores_rrhh,periodos_informes_rrhh,valores_informe_rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[entidades_rrhh, indicadores_rrhh, periodos_in..."
4,rrhh,indicadores_rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,"[indicadores_rsc, rrhh_altas, rrhh_bajas, r_rh..."


In [79]:
test_file['tables']

0    [plantilla_mensual, tipo_contrato, categoria_p...
1    [plantilla_mensual, brecha_salarios, tipo_cont...
2    [rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh...
3    [entidades_rrhh, indicadores_rrhh, periodos_in...
4    [indicadores_rsc, rrhh_altas, rrhh_bajas, r_rh...
5    [brecha_salarial, brecha_ajustada, ids_a_exclu...
6    [plantilla_anual, movimientos_plantilla_unica,...
7    [d_hs_causa, tipo_contrato, d_hs_empresa, d_hs...
8    [canal_gestion_responsable, criticidad, delega...
9    [gen_cuestionario, gen_ambito1, gestdoc_ambito...
Name: tables, dtype: object

We drop the individual table name columns in order to get a df with only the project name and the tables list

In [80]:
test_file.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 'tables'], dtype='object')

In [81]:
project_with_tables = test_file.drop(columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [82]:
project_with_tables_nan = project_with_tables.rename(columns={0:'Project'})

In [83]:
project_with_tables_nan.head()

,Project,tables
0,rrhh,"[plantilla_mensual, tipo_contrato, categoria_p..."
1,brecha_salarial,"[plantilla_mensual, brecha_salarios, tipo_cont..."
2,rrhh,"[rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh..."
3,rrhh,"[entidades_rrhh, indicadores_rrhh, periodos_in..."
4,rrhh,"[indicadores_rsc, rrhh_altas, rrhh_bajas, r_rh..."


In [84]:
project_with_tables_nan['tables']

0    [plantilla_mensual, tipo_contrato, categoria_p...
1    [plantilla_mensual, brecha_salarios, tipo_cont...
2    [rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh...
3    [entidades_rrhh, indicadores_rrhh, periodos_in...
4    [indicadores_rsc, rrhh_altas, rrhh_bajas, r_rh...
5    [brecha_salarial, brecha_ajustada, ids_a_exclu...
6    [plantilla_anual, movimientos_plantilla_unica,...
7    [d_hs_causa, tipo_contrato, d_hs_empresa, d_hs...
8    [canal_gestion_responsable, criticidad, delega...
9    [gen_cuestionario, gen_ambito1, gestdoc_ambito...
Name: tables, dtype: object

In [85]:
project_with_tables_nan['tables'].isnull().values.any()

False

Creating a dictionary for projects and table names where index will be the type of project and the values the list of tables

In [86]:
tables_dict_nan = {}

for index, row in project_with_tables_nan.iterrows():
    tables_dict_nan[index] = row[1]

In [87]:
tables_dict_nan

{0: ['plantilla_mensual',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 1: ['plantilla_mensual',
  'brecha_salarios',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 2: ['rrhh_absentismo',
  'rrhh_altas',
  'rrhh_bajas',
  'rhhh_plantilla',
  'rrhh_contratacion',
  'rrhh_indicadores',
  'rrhh_jornada_trabajo',
  'rrhhh_datos',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 3: ['entidades_rrhh',
  'indicadores_rrhh',
  'periodos_informes_rrhh',
  'valores_informe_rrhh',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 4: ['indicadores_rsc',
  'rrhh_altas',
  'rrhh_bajas',
  'r_rhh_tipo_entidad_juridica',
  'responsables_de_sociedades_juridicas',
  'rrhh_carga_indicadores',
  'rrhh_carga_indicadores_mes',
  nan,
  'u_rhh_carga_subactividades',

In [88]:
project_id_dict = {}

for index, row in project_with_tables_nan.iterrows():
    project_id_dict[index] = row[0]

In [89]:
project_id_dict

{0: 'rrhh',
 1: 'brecha_salarial',
 2: 'rrhh',
 3: 'rrhh',
 4: 'rrhh',
 5: 'brecha_salarial',
 6: 'rrhh',
 7: 'seguridad_y_salud',
 8: 'canal_negocio_responsable',
 9: 'operaciones'}

We remove all the NaN elements in the list inside the dictionary

In [90]:
tablesdict = {}
count=0
for list_ in tables_dict_nan.values():
    vector=[]
    
    for item in list_:
        
        if str(item) != 'nan':
            vector.append(str(item))
   
            
    tablesdict[count]= vector
    count+=1  

In [91]:
tablesdict

{0: ['plantilla_mensual',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo'],
 1: ['plantilla_mensual',
  'brecha_salarios',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo'],
 2: ['rrhh_absentismo',
  'rrhh_altas',
  'rrhh_bajas',
  'rhhh_plantilla',
  'rrhh_contratacion',
  'rrhh_indicadores',
  'rrhh_jornada_trabajo',
  'rrhhh_datos'],
 3: ['entidades_rrhh',
  'indicadores_rrhh',
  'periodos_informes_rrhh',
  'valores_informe_rrhh'],
 4: ['indicadores_rsc',
  'rrhh_altas',
  'rrhh_bajas',
  'r_rhh_tipo_entidad_juridica',
  'responsables_de_sociedades_juridicas',
  'rrhh_carga_indicadores',
  'rrhh_carga_indicadores_mes',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades'],
 5: ['brecha_salarial',
  'brecha_ajustada',
  'ids_a_excluir_

We create a df with all types of tables

In [92]:
all_tables = pd.DataFrame()
vector = []

for list_ in tablesdict.values():
   for item in list_: 
    vector.append(item)


In [93]:
all_tables = all_tables.append(vector)

all_tables

,0
0,plantilla_mensual
1,tipo_contrato
2,categoria_profesional
3,empleados_excluidos
4,nacionalidad
...,...
72,preguntas
73,respuestas
74,resultados
75,planes_de_accion


In [94]:
all_tables = all_tables.drop_duplicates()

In [95]:
id_tables = {}

count=0

for index, row in all_tables.iterrows():
  id_tables[count] = row[0]
  count+=1

In [96]:
id_tables

{0: 'plantilla_mensual',
 1: 'tipo_contrato',
 2: 'categoria_profesional',
 3: 'empleados_excluidos',
 4: 'nacionalidad',
 5: 'pais_trabajo',
 6: 'brecha_salarios',
 7: 'rrhh_absentismo',
 8: 'rrhh_altas',
 9: 'rrhh_bajas',
 10: 'rhhh_plantilla',
 11: 'rrhh_contratacion',
 12: 'rrhh_indicadores',
 13: 'rrhh_jornada_trabajo',
 14: 'rrhhh_datos',
 15: 'entidades_rrhh',
 16: 'indicadores_rrhh',
 17: 'periodos_informes_rrhh',
 18: 'valores_informe_rrhh',
 19: 'indicadores_rsc',
 20: 'r_rhh_tipo_entidad_juridica',
 21: 'responsables_de_sociedades_juridicas',
 22: 'rrhh_carga_indicadores',
 23: 'rrhh_carga_indicadores_mes',
 24: 'u_rhh_carga_subactividades',
 25: 'brecha_salarial',
 26: 'brecha_ajustada',
 27: 'ids_a_excluir_brecha',
 28: 'percentiles_brecha_salarial',
 29: 'matriculas_a_obviar_brecha',
 30: 'permisos_brecha_salarial',
 31: 'import_brecha',
 32: 'plantilla_anual',
 33: 'movimientos_plantilla_unica',
 34: 'movimientos_cierre_mes',
 35: 'tipo_de_modificacion',
 36: 'versiones_

In [97]:
# Define a function that retrieves all the ids and names for tables containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in tables name
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in tables name, and an empty list otherwise.
    """
    # convert input text to lowercase

    text_ = text.lower()
    
    # find occurances
    search = [(k, v .lower().find(text_)) for k, v in list(ids.items())]
    
    # Get the IDs corresponding to the given text
    index = [k for k, v in search if v > -1]
    
    # Return a list with the id and the name
    out = []
    for i in index:
        out.append((i, ids[i]))
        
    
    return out

In [98]:
returnItemId('altas', id_tables)

[(8, 'rrhh_altas')]

In [99]:
print(len(set(id_tables.keys())))
print(len(set(id_tables.values())))

62
62


We create a df with the dictionary with the columns as the index of the projects

In [100]:
projects = pd.DataFrame.from_dict(tables_dict_nan, orient='index').reset_index()

projects

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,3,entidades_rrhh,indicadores_rrhh,periodos_informes_rrhh,valores_informe_rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,indicadores_rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades
5,5,brecha_salarial,brecha_ajustada,ids_a_excluir_brecha,percentiles_brecha_salarial,matriculas_a_obviar_brecha,permisos_brecha_salarial,import_brecha,rrhh_altas,NaN,NaN,NaN,NaN,NaN,NaN
6,6,plantilla_anual,movimientos_plantilla_unica,movimientos_cierre_mes,tipo_de_modificacion,versiones_plantilla_unica,sbq_movimientos_periodo_anterios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,d_hs_causa,tipo_contrato,d_hs_empresa,d_hs_presentado,d_hs_tareas,d_hs_territorio,d_hs_tipo_accidente_externo,d_hs_unidad,t_hs_accidentes_externos,t_hs_comunicacion_investigacion_accidentes,NaN,NaN,NaN,NaN
8,8,canal_gestion_responsable,criticidad,delegaciones,emails_del_gestor,logs_de_emails,plantillas_emails,tipologias_de_consultas_comerciales,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,gen_cuestionario,gen_ambito1,gestdoc_ambito2,preguntas,respuestas,resultados,planes_de_accion,palancas,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
dummies_projects = pd.get_dummies(projects)

dummies_projects

,index,0_brecha_salarial,0_canal_gestion_responsable,0_d_hs_causa,0_entidades_rrhh,0_gen_cuestionario,0_indicadores_rsc,0_plantilla_anual,0_plantilla_mensual,0_rrhh_absentismo,1_brecha_ajustada,1_brecha_salarios,1_criticidad,1_gen_ambito1,1_indicadores_rrhh,1_movimientos_plantilla_unica,1_rrhh_altas,1_tipo_contrato,2_categoria_profesional,2_d_hs_empresa,2_delegaciones,2_gestdoc_ambito2,2_ids_a_excluir_brecha,2_movimientos_cierre_mes,2_periodos_informes_rrhh,2_rrhh_bajas,2_tipo_contrato,3_categoria_profesional,3_d_hs_presentado,3_emails_del_gestor,3_empleados_excluidos,3_percentiles_brecha_salarial,3_preguntas,3_r_rhh_tipo_entidad_juridica,3_rhhh_plantilla,3_tipo_de_modificacion,3_valores_informe_rrhh,4_d_hs_tareas,4_empleados_excluidos,4_logs_de_emails,4_matriculas_a_obviar_brecha,4_nacionalidad,4_responsables_de_sociedades_juridicas,4_respuestas,4_rrhh_contratacion,4_versiones_plantilla_unica,5_d_hs_territorio,5_nacionalidad,5_pais_trabajo,5_permisos_brecha_salarial,5_plantillas_emails,5_resultados,5_rrhh_carga_indicadores,5_rrhh_indicadores,5_sbq_movimientos_periodo_anterios,6_d_hs_tipo_accidente_externo,6_import_brecha,6_pais_trabajo,6_planes_de_accion,6_rrhh_carga_indicadores_mes,6_rrhh_jornada_trabajo,6_tipologias_de_consultas_comerciales,7_d_hs_unidad,7_palancas,7_rrhh_altas,7_rrhhh_datos,8_t_hs_accidentes_externos,8_u_rhh_carga_subactividades,9_t_hs_comunicacion_investigacion_accidentes,9_u_rhh_carga_subactividades,10_u_rhh_carga_subactividades,11_u_rhh_carga_subactividades,12_u_rhh_carga_subactividades,13_u_rhh_carga_subactividades
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,1,1
5,5,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0
8,8,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [102]:
projects_reset = pd.DataFrame.from_dict(tables_dict_nan)

projects_reset

,0,1,2,3,4,5,6,7,8,9
0,plantilla_mensual,plantilla_mensual,rrhh_absentismo,entidades_rrhh,indicadores_rsc,brecha_salarial,plantilla_anual,d_hs_causa,canal_gestion_responsable,gen_cuestionario
1,tipo_contrato,brecha_salarios,rrhh_altas,indicadores_rrhh,rrhh_altas,brecha_ajustada,movimientos_plantilla_unica,tipo_contrato,criticidad,gen_ambito1
2,categoria_profesional,tipo_contrato,rrhh_bajas,periodos_informes_rrhh,rrhh_bajas,ids_a_excluir_brecha,movimientos_cierre_mes,d_hs_empresa,delegaciones,gestdoc_ambito2
3,empleados_excluidos,categoria_profesional,rhhh_plantilla,valores_informe_rrhh,r_rhh_tipo_entidad_juridica,percentiles_brecha_salarial,tipo_de_modificacion,d_hs_presentado,emails_del_gestor,preguntas
4,nacionalidad,empleados_excluidos,rrhh_contratacion,NaN,responsables_de_sociedades_juridicas,matriculas_a_obviar_brecha,versiones_plantilla_unica,d_hs_tareas,logs_de_emails,respuestas
5,pais_trabajo,nacionalidad,rrhh_indicadores,NaN,rrhh_carga_indicadores,permisos_brecha_salarial,sbq_movimientos_periodo_anterios,d_hs_territorio,plantillas_emails,resultados
6,NaN,pais_trabajo,rrhh_jornada_trabajo,NaN,rrhh_carga_indicadores_mes,import_brecha,NaN,d_hs_tipo_accidente_externo,tipologias_de_consultas_comerciales,planes_de_accion
7,NaN,NaN,rrhhh_datos,NaN,NaN,rrhh_altas,NaN,d_hs_unidad,NaN,palancas
8,NaN,NaN,NaN,NaN,u_rhh_carga_subactividades,NaN,NaN,t_hs_accidentes_externos,NaN,NaN
9,NaN,NaN,NaN,NaN,u_rhh_carga_subactividades,NaN,NaN,t_hs_comunicacion_investigacion_accidentes,NaN,NaN


In [103]:
projects_array = projects.iloc[:, :].values

In [104]:
projects_array

array([[0, 'plantilla_mensual', 'tipo_contrato', 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan, nan],
       [1, 'plantilla_mensual', 'brecha_salarios', 'tipo_contrato',
        'categoria_profesional', 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan],
       [2, 'rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas',
        'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores',
        'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan,
        nan],
       [3, 'entidades_rrhh', 'indicadores_rrhh',
        'periodos_informes_rrhh', 'valores_informe_rrhh', nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan],
       [4, 'indicadores_rsc', 'rrhh_altas', 'rrhh_bajas',
        'r_rhh_tipo_entidad_juridica',
        'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactivid

In [105]:
from sklearn.preprocessing import LabelEncoder

In [106]:
table = LabelEncoder()

In [107]:
projects_array

array([[0, 'plantilla_mensual', 'tipo_contrato', 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan, nan],
       [1, 'plantilla_mensual', 'brecha_salarios', 'tipo_contrato',
        'categoria_profesional', 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan],
       [2, 'rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas',
        'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores',
        'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan,
        nan],
       [3, 'entidades_rrhh', 'indicadores_rrhh',
        'periodos_informes_rrhh', 'valores_informe_rrhh', nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan],
       [4, 'indicadores_rsc', 'rrhh_altas', 'rrhh_bajas',
        'r_rhh_tipo_entidad_juridica',
        'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactivid

In [108]:
projects_array[:, 1]= table.fit_transform(projects_array[:, 1]) #vectorizamos la primera columna

In [109]:
projects_array[:,3]= table.fit_transform(projects_array[:,3])

In [110]:
projects_array[:,2:]

array([['tipo_contrato', 0, 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan],
       ['brecha_salarios', 8, 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan],
       ['rrhh_altas', 7, 'rhhh_plantilla', 'rrhh_contratacion',
        'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan,
        nan, nan, nan, nan, nan],
       ['indicadores_rrhh', 6, 'valores_informe_rrhh', nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan],
       ['rrhh_altas', 7, 'r_rhh_tipo_entidad_juridica',
        'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades'],
       ['brecha_ajustada', 4, 'percentiles_b

In [111]:
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
project_with_tables_nan

,Project,tables
0,rrhh,"[plantilla_mensual, tipo_contrato, categoria_p..."
1,brecha_salarial,"[plantilla_mensual, brecha_salarios, tipo_cont..."
2,rrhh,"[rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh..."
3,rrhh,"[entidades_rrhh, indicadores_rrhh, periodos_in..."
4,rrhh,"[indicadores_rsc, rrhh_altas, rrhh_bajas, r_rh..."
5,brecha_salarial,"[brecha_salarial, brecha_ajustada, ids_a_exclu..."
6,rrhh,"[plantilla_anual, movimientos_plantilla_unica,..."
7,seguridad_y_salud,"[d_hs_causa, tipo_contrato, d_hs_empresa, d_hs..."
8,canal_negocio_responsable,"[canal_gestion_responsable, criticidad, delega..."
9,operaciones,"[gen_cuestionario, gen_ambito1, gestdoc_ambito..."


In [113]:
##convertimos todos los valores de la lista de nombre de tablas a str para poder pasarselo al countvectorizer

In [114]:
project_with_tables_nan['tables'] = project_with_tables_nan['tables'].astype('str')

In [115]:
for list_ in project_with_tables_nan['tables']:
  print(list_)
  print((list_.split()))

['plantilla_mensual', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan]
["['plantilla_mensual',", "'tipo_contrato',", "'categoria_profesional',", "'empleados_excluidos',", "'nacionalidad',", "'pais_trabajo',", 'nan,', 'nan,', 'nan,', 'nan,', 'nan,', 'nan,', 'nan,', 'nan]']
['plantilla_mensual', 'brecha_salarios', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan]
["['plantilla_mensual',", "'brecha_salarios',", "'tipo_contrato',", "'categoria_profesional',", "'empleados_excluidos',", "'nacionalidad',", "'pais_trabajo',", 'nan,', 'nan,', 'nan,', 'nan,', 'nan,', 'nan,', 'nan]']
['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas', 'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan, nan]
["['rrhh_absentismo',", "'rrhh_altas',", "'rrhh_bajas',", "'rhhh_plantilla

We are going to generate a sparse matrix from our tables with CountVectorizer from Sklearn

In [116]:
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
cv = CountVectorizer()

In [118]:
X = cv.fit_transform(project_with_tables_nan['tables'])

In [119]:
project_with_tables_nan['tables']

0    ['plantilla_mensual', 'tipo_contrato', 'catego...
1    ['plantilla_mensual', 'brecha_salarios', 'tipo...
2    ['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas'...
3    ['entidades_rrhh', 'indicadores_rrhh', 'period...
4    ['indicadores_rsc', 'rrhh_altas', 'rrhh_bajas'...
5    ['brecha_salarial', 'brecha_ajustada', 'ids_a_...
6    ['plantilla_anual', 'movimientos_plantilla_uni...
7    ['d_hs_causa', 'tipo_contrato', 'd_hs_empresa'...
8    ['canal_gestion_responsable', 'criticidad', 'd...
9    ['gen_cuestionario', 'gen_ambito1', 'gestdoc_a...
Name: tables, dtype: object

In [120]:
print(cv.get_feature_names())

['brecha_ajustada', 'brecha_salarial', 'brecha_salarios', 'canal_gestion_responsable', 'categoria_profesional', 'criticidad', 'd_hs_causa', 'd_hs_empresa', 'd_hs_presentado', 'd_hs_tareas', 'd_hs_territorio', 'd_hs_tipo_accidente_externo', 'd_hs_unidad', 'delegaciones', 'emails_del_gestor', 'empleados_excluidos', 'entidades_rrhh', 'gen_ambito1', 'gen_cuestionario', 'gestdoc_ambito2', 'ids_a_excluir_brecha', 'import_brecha', 'indicadores_rrhh', 'indicadores_rsc', 'logs_de_emails', 'matriculas_a_obviar_brecha', 'movimientos_cierre_mes', 'movimientos_plantilla_unica', 'nacionalidad', 'nan', 'pais_trabajo', 'palancas', 'percentiles_brecha_salarial', 'periodos_informes_rrhh', 'permisos_brecha_salarial', 'planes_de_accion', 'plantilla_anual', 'plantilla_mensual', 'plantillas_emails', 'preguntas', 'r_rhh_tipo_entidad_juridica', 'responsables_de_sociedades_juridicas', 'respuestas', 'resultados', 'rhhh_plantilla', 'rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas', 'rrhh_carga_indicadores', 'rrhh_ca



show resulting vocabulary; the numbers are not counts, they are the position in the sparse vector.


In [121]:
cv.vocabulary_ 


{'brecha_ajustada': 0,
 'brecha_salarial': 1,
 'brecha_salarios': 2,
 'canal_gestion_responsable': 3,
 'categoria_profesional': 4,
 'criticidad': 5,
 'd_hs_causa': 6,
 'd_hs_empresa': 7,
 'd_hs_presentado': 8,
 'd_hs_tareas': 9,
 'd_hs_territorio': 10,
 'd_hs_tipo_accidente_externo': 11,
 'd_hs_unidad': 12,
 'delegaciones': 13,
 'emails_del_gestor': 14,
 'empleados_excluidos': 15,
 'entidades_rrhh': 16,
 'gen_ambito1': 17,
 'gen_cuestionario': 18,
 'gestdoc_ambito2': 19,
 'ids_a_excluir_brecha': 20,
 'import_brecha': 21,
 'indicadores_rrhh': 22,
 'indicadores_rsc': 23,
 'logs_de_emails': 24,
 'matriculas_a_obviar_brecha': 25,
 'movimientos_cierre_mes': 26,
 'movimientos_plantilla_unica': 27,
 'nacionalidad': 28,
 'nan': 29,
 'pais_trabajo': 30,
 'palancas': 31,
 'percentiles_brecha_salarial': 32,
 'periodos_informes_rrhh': 33,
 'permisos_brecha_salarial': 34,
 'planes_de_accion': 35,
 'plantilla_anual': 36,
 'plantilla_mensual': 37,
 'plantillas_emails': 38,
 'preguntas': 39,
 'r_rhh_t

In [122]:
X.shape ##Shape of our matriz

(10, 63)

In [123]:
print(X.toarray())

[[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
   0  0  0  0  1  8  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
   0  0  0  0  1  7  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1
   0  0  1  1  1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0
   0  0  0  0  0 10  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
   0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  1  1
   1  1  0  0  0  0  0  0  0  0  0  0  6  0  0]
 [ 1  1  0  0  0  0 

In [124]:
df_sparse_matrix = pd.DataFrame(data = X.toarray(), index = project_with_tables_nan['tables'], columns=cv.get_feature_names())
df_sparse_matrix


,brecha_ajustada,brecha_salarial,brecha_salarios,canal_gestion_responsable,categoria_profesional,criticidad,d_hs_causa,d_hs_empresa,d_hs_presentado,d_hs_tareas,d_hs_territorio,d_hs_tipo_accidente_externo,d_hs_unidad,delegaciones,emails_del_gestor,empleados_excluidos,entidades_rrhh,gen_ambito1,gen_cuestionario,gestdoc_ambito2,ids_a_excluir_brecha,import_brecha,indicadores_rrhh,indicadores_rsc,logs_de_emails,matriculas_a_obviar_brecha,movimientos_cierre_mes,movimientos_plantilla_unica,nacionalidad,nan,pais_trabajo,palancas,percentiles_brecha_salarial,periodos_informes_rrhh,permisos_brecha_salarial,planes_de_accion,plantilla_anual,plantilla_mensual,plantillas_emails,preguntas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,respuestas,resultados,rhhh_plantilla,rrhh_absentismo,rrhh_altas,rrhh_bajas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,sbq_movimientos_periodo_anterios,t_hs_accidentes_externos,t_hs_comunicacion_investigacion_accidentes,tipo_contrato,tipo_de_modificacion,tipologias_de_consultas_comerciales,u_rhh_carga_subactividades,valores_informe_rrhh,versiones_plantilla_unica
tables,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"['plantilla_mensual', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,8,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
"['plantilla_mensual', 'brecha_salarios', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan]",0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,7,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
"['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas', 'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0
"['entidades_rrhh', 'indicadores_rrhh', 'periodos_informes_rrhh', 'valores_informe_rrhh', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
"['indicadores_rsc', 'rrhh_altas', 'rrhh_bajas', 'r_rhh_tipo_entidad_juridica', 'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores', 'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,6,0,0
"['brecha_salarial', 'brecha_ajustada', 'ids_a_excluir_brecha', 'percentiles_brecha_salarial', 'matriculas_a_obviar_brecha', 'permisos_brecha_salarial', 'import_brecha', 'rrhh_altas', nan, nan, nan, nan, nan, nan]",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,6,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"['plantilla_anual', 'movimientos_plantilla_unica', 'movimientos_cierre_mes', 'tipo_de_modificacion', 'versiones_plantilla_unica', 'sbq_movimientos_periodo_anterios', nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
"['d_hs_causa', 'tipo_contrato', 'd_hs_empresa', 'd_hs_presentado', 'd_hs_tareas', 'd_hs_territorio', 'd_hs_tipo_accidente_externo', 'd_hs_unidad', 't_hs_accidentes_externos', 't_hs_comunicacion_investigacion_accidentes', nan, nan, nan, nan]",0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0
"['canal_gestion_responsable', 'criticidad', 'delega

In [125]:
df_sparse_matrix.columns.tolist()

['brecha_ajustada',
 'brecha_salarial',
 'brecha_salarios',
 'canal_gestion_responsable',
 'categoria_profesional',
 'criticidad',
 'd_hs_causa',
 'd_hs_empresa',
 'd_hs_presentado',
 'd_hs_tareas',
 'd_hs_territorio',
 'd_hs_tipo_accidente_externo',
 'd_hs_unidad',
 'delegaciones',
 'emails_del_gestor',
 'empleados_excluidos',
 'entidades_rrhh',
 'gen_ambito1',
 'gen_cuestionario',
 'gestdoc_ambito2',
 'ids_a_excluir_brecha',
 'import_brecha',
 'indicadores_rrhh',
 'indicadores_rsc',
 'logs_de_emails',
 'matriculas_a_obviar_brecha',
 'movimientos_cierre_mes',
 'movimientos_plantilla_unica',
 'nacionalidad',
 'nan',
 'pais_trabajo',
 'palancas',
 'percentiles_brecha_salarial',
 'periodos_informes_rrhh',
 'permisos_brecha_salarial',
 'planes_de_accion',
 'plantilla_anual',
 'plantilla_mensual',
 'plantillas_emails',
 'preguntas',
 'r_rhh_tipo_entidad_juridica',
 'responsables_de_sociedades_juridicas',
 'respuestas',
 'resultados',
 'rhhh_plantilla',
 'rrhh_absentismo',
 'rrhh_altas',
 '

We drop all NaN values from our matrix

In [126]:
df_sparse_matrix.drop(columns={'nan'}, inplace=True) 

If we sum all the values in a row we obtain the number of tables the project have

In [127]:


cols_to_sum = df_sparse_matrix.columns[ : df_sparse_matrix.shape[1]-1]

df_sparse_matrix['Sum Total'] = df_sparse_matrix[cols_to_sum].sum(axis=1)

df_sparse_matrix

,brecha_ajustada,brecha_salarial,brecha_salarios,canal_gestion_responsable,categoria_profesional,criticidad,d_hs_causa,d_hs_empresa,d_hs_presentado,d_hs_tareas,d_hs_territorio,d_hs_tipo_accidente_externo,d_hs_unidad,delegaciones,emails_del_gestor,empleados_excluidos,entidades_rrhh,gen_ambito1,gen_cuestionario,gestdoc_ambito2,ids_a_excluir_brecha,import_brecha,indicadores_rrhh,indicadores_rsc,logs_de_emails,matriculas_a_obviar_brecha,movimientos_cierre_mes,movimientos_plantilla_unica,nacionalidad,pais_trabajo,palancas,percentiles_brecha_salarial,periodos_informes_rrhh,permisos_brecha_salarial,planes_de_accion,plantilla_anual,plantilla_mensual,plantillas_emails,preguntas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,respuestas,resultados,rhhh_plantilla,rrhh_absentismo,rrhh_altas,rrhh_bajas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,sbq_movimientos_periodo_anterios,t_hs_accidentes_externos,t_hs_comunicacion_investigacion_accidentes,tipo_contrato,tipo_de_modificacion,tipologias_de_consultas_comerciales,u_rhh_carga_subactividades,valores_informe_rrhh,versiones_plantilla_unica,Sum Total
tables,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"['plantilla_mensual', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,6
"['plantilla_mensual', 'brecha_salarios', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan]",0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7
"['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas', 'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,8
"['entidades_rrhh', 'indicadores_rrhh', 'periodos_informes_rrhh', 'valores_informe_rrhh', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4
"['indicadores_rsc', 'rrhh_altas', 'rrhh_bajas', 'r_rhh_tipo_entidad_juridica', 'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores', 'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades']",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,6,0,0,13
"['brecha_salarial', 'brecha_ajustada', 'ids_a_excluir_brecha', 'percentiles_brecha_salarial', 'matriculas_a_obviar_brecha', 'permisos_brecha_salarial', 'import_brecha', 'rrhh_altas', nan, nan, nan, nan, nan, nan]",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
"['plantilla_anual', 'movimientos_plantilla_unica', 'movimientos_cierre_mes', 'tipo_de_modificacion', 'versiones_plantilla_unica', 'sbq_movimientos_periodo_anterios', nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,5
"['d_hs_causa', 'tipo_contrato', 'd_hs_empresa', 'd_hs_presentado', 'd_hs_tareas', 'd_hs_territorio', 'd_hs_tipo_accidente_externo', 'd_hs_unidad', 't_hs_accidentes_externos', 't_hs_comunicacion_investigacion_accidentes', nan, nan, nan, nan]",0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,10
"['canal_gestion_responsable', 'criticidad', 

We sum all columns and transpose the df to obtain the number of times a table is repeated in all projects

In [128]:
most_repeated_tables = pd.DataFrame(df_sparse_matrix.sum(axis=0)).T

most_repeated_tables


,brecha_ajustada,brecha_salarial,brecha_salarios,canal_gestion_responsable,categoria_profesional,criticidad,d_hs_causa,d_hs_empresa,d_hs_presentado,d_hs_tareas,d_hs_territorio,d_hs_tipo_accidente_externo,d_hs_unidad,delegaciones,emails_del_gestor,empleados_excluidos,entidades_rrhh,gen_ambito1,gen_cuestionario,gestdoc_ambito2,ids_a_excluir_brecha,import_brecha,indicadores_rrhh,indicadores_rsc,logs_de_emails,matriculas_a_obviar_brecha,movimientos_cierre_mes,movimientos_plantilla_unica,nacionalidad,pais_trabajo,palancas,percentiles_brecha_salarial,periodos_informes_rrhh,permisos_brecha_salarial,planes_de_accion,plantilla_anual,plantilla_mensual,plantillas_emails,preguntas,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,respuestas,resultados,rhhh_plantilla,rrhh_absentismo,rrhh_altas,rrhh_bajas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,sbq_movimientos_periodo_anterios,t_hs_accidentes_externos,t_hs_comunicacion_investigacion_accidentes,tipo_contrato,tipo_de_modificacion,tipologias_de_consultas_comerciales,u_rhh_carga_subactividades,valores_informe_rrhh,versiones_plantilla_unica,Sum Total
0,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,3,2,1,1,1,1,1,1,1,1,1,3,1,1,6,1,1,76


Now the idea is to filter our dataset from the user response of what type of project is needed, and recommend the most repeated tables and if there aren't any option repeated recommend something. As this system is going to be feeded from the users response is going to be in continuous grow. Each option a user selects is going to be a new project with new tables

First we are going to obtain the mean number of tables a project have in order to see how many tables we need to present as minimun result

In [129]:
mean_tables = pd.DataFrame(df_sparse_matrix.mean(axis=0)).T

mean_tables = mean_tables['Sum Total']

mean_tables #The minimun amount of tables we have to recommend is 7

0    7.6
Name: Sum Total, dtype: float64

Input from the user and filter our dataset by the project

In [130]:
user_choice = input('Select type of project ')

Select type of project rrhh


In [131]:
user_choice

'rrhh'

In [133]:
project_filtered_nan = project_with_tables_nan[project_with_tables_nan['Project'] == user_choice]

In [134]:
project_filtered_nan

,Project,tables
0,rrhh,"['plantilla_mensual', 'tipo_contrato', 'catego..."
2,rrhh,"['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas'..."
3,rrhh,"['entidades_rrhh', 'indicadores_rrhh', 'period..."
4,rrhh,"['indicadores_rsc', 'rrhh_altas', 'rrhh_bajas'..."
6,rrhh,"['plantilla_anual', 'movimientos_plantilla_uni..."


We repeate same stages we did before to obtain the most repeated tables for 'rrhh' project

In [138]:
cv = CountVectorizer()
X = cv.fit_transform(project_filtered_nan['tables'])
df_sparse_matrix = pd.DataFrame(data = X.toarray(), index = project_filtered_nan['tables'], columns=cv.get_feature_names())
df_sparse_matrix.drop(columns={'nan'}, inplace=True) 
df_sparse_matrix



,categoria_profesional,empleados_excluidos,entidades_rrhh,indicadores_rrhh,indicadores_rsc,movimientos_cierre_mes,movimientos_plantilla_unica,nacionalidad,pais_trabajo,periodos_informes_rrhh,plantilla_anual,plantilla_mensual,r_rhh_tipo_entidad_juridica,responsables_de_sociedades_juridicas,rhhh_plantilla,rrhh_absentismo,rrhh_altas,rrhh_bajas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,sbq_movimientos_periodo_anterios,tipo_contrato,tipo_de_modificacion,u_rhh_carga_subactividades,valores_informe_rrhh,versiones_plantilla_unica
tables,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"['plantilla_mensual', 'tipo_contrato', 'categoria_profesional', 'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan]",1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
"['rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas', 'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan, nan]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0
"['entidades_rrhh', 'indicadores_rrhh', 'periodos_informes_rrhh', 'valores_informe_rrhh', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
"['indicadores_rsc', 'rrhh_altas', 'rrhh_bajas', 'r_rhh_tipo_entidad_juridica', 'responsables_de_sociedades_juridicas', 'rrhh_carga_indicadores', 'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades']",0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,6,0,0
"['plantilla_anual', 'movimientos_plantilla_unica', 'movimientos_cierre_mes', 'tipo_de_modificacion', 'versiones_plantilla_unica', 'sbq_movimientos_periodo_anterios', nan, nan, nan, nan, nan, nan, nan, nan]",0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1


In [148]:
most_repeated_tables = pd.DataFrame(df_sparse_matrix.sum(axis=0))

most_repeated_tables.rename(columns={0:'Num'}, inplace=True)

In [165]:

most_repeated_tables  = most_repeated_tables.sort_values(by = ['Num'],ascending=False)
most_repeated_tables = most_repeated_tables.reset_index()

most_repeated_tables

,level_0,index,Num
0,0,u_rhh_carga_subactividades,6
1,2,rrhh_bajas,2
2,1,rrhh_altas,2
3,16,rrhh_carga_indicadores,1
4,28,rrhh_absentismo,1
5,27,categoria_profesional,1
6,26,empleados_excluidos,1
7,25,valores_informe_rrhh,1
8,24,tipo_de_modificacion,1
9,23,tipo_contrato,1


We get seven tables we would suggest to the user

In [174]:
seven_tables = most_repeated_tables.iloc[:8]

seven_tables.drop(columns=['level_0'], inplace=True)

seven_tables

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,Num
0,u_rhh_carga_subactividades,6
1,rrhh_bajas,2
2,rrhh_altas,2
3,rrhh_carga_indicadores,1
4,rrhh_absentismo,1
5,categoria_profesional,1
6,empleados_excluidos,1
7,valores_informe_rrhh,1
